In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from flarestack import analyse, ResultsHandler
from flarestack.data.simulate.potemkin import potemkin_dataset, IdealBackgroundFluxModel
from flarestack.utils.prepare_catalogue import ps_catalogue_name
from flarestack.data.public import icecube_ps_3_year
from flarestack.shared import flux_to_k
import logging
logging.getLogger().setLevel("INFO")

In [2]:
# We create an imaginary dataset season, between 50000 MJD and 55000 MJD, with 100% uptime

bkg_time_pdf_dict = {
    "time_pdf_name": "fixed_ref_box",
    "fixed_ref_time_mjd": 50000,
    "pre_window": 0.,
    "post_window": 500.,
    "livetime": True
}

# We take the rough normalisation of the atmospheric flux, ~10^8 GeV cm^-2 s^-1 sr^-1 at ~200 TeV

f_200tev = (10. ** -8.)
flux_norm_1_gev = f_200tev * ((2.e5) ** 1.7)

bkg_flux_model = IdealBackgroundFluxModel(
    flux_norm_1_gev,
    bkg_time_pdf_dict=bkg_time_pdf_dict,
)

# We create a simulated dataset, containg only background, using the published effective areas for IceCube with 86 strings 2012

potemkin_dataset.set_sim_params(
    name="IC86-2012",
    bkg_flux_model=bkg_flux_model,
#     resimulate=True
)

INFO:root:Minimum Energy is 100.0 GeV.
INFO:root:Maximum Energy is 10000000.0 GeV
INFO:root:Found existing simulation at /Users/robertstein/flarestack__data/input/sim_datasets/SimCube_default/IC86-2012/10274270429.662216//50000.0_50500.0.npy


In [3]:
# Setup analysis, injecting an E^-2 source

inj_dict = {
    "injection_sig_time_pdf": {
        "time_pdf_name": "steady"
    },
    "injection_energy_pdf": {
        "energy_pdf_name": "power_law",
        "gamma": 3.7
    }
}

# Perform a "standard" analysis (fitting spectral index and ns)

llh_dict = {
    "llh_name": "spatial",
    "llh_sig_time_pdf": {
        "time_pdf_name": "steady"
    },
    "llh_bkg_time_pdf":  {
        "time_pdf_name": "steady"
    },
#     "llh_energy_pdf": {
#         "energy_pdf_name": "power_law"
#     }
}

# Analysis with "source catalogue" consisting of a single source at sindec=0.5

mh_dict = {
            "name": "examples/analysis_with_sim",
            "mh_name": "fixed_weights",
#             "dataset": potemkin_dataset.get_seasons("IC86-2012"),
            "dataset": icecube_ps_3_year.get_seasons("IC86-2012"),
            "catalogue": ps_catalogue_name(0.5),
            "inj_dict": inj_dict,
            "llh_dict": llh_dict,
            "n_steps": 5,
            "n_trials": 10,
            "scale": 800000.
        }

In [4]:
print(potemkin_dataset.get_single_season("IC86-2012"))

In [5]:
analyse(mh_dict, n_cpu=2, cluster=False)

rh = ResultsHandler(mh_dict)

INFO:root:Using 'spatial' LLH class
INFO:root:Initialising Injector for IC86-2012
INFO:root:Using 'spatial' LLH class
INFO:root:Using 'spatial' LLH class
INFO:root:Added 140 trials to queue. Now processing.
INFO: 2020-05-10 16:36:46,144 - 97324 - Using 'spatial' LLH class
INFO: 2020-05-10 16:36:46,147 - 97325 - Using 'spatial' LLH class
INFO:root:140 tasks remaining.


n_coincident 347
n_coincident 362
n_coincident 355
n_coincident 367
n_coincident 362
n_coincident 390
n_coincident 335
n_coincident 353
n_coincident 399
n_coincident 350
n_coincident 376
n_coincident 359
n_coincident 388
n_coincident 363
n_coincident 343
n_coincident 388
n_coincident 405
n_coincident 336
n_coincident 376
n_coincident 377
n_coincident 339
n_coincident 358
n_coincident 397
n_coincident 345
n_coincident 350
n_coincident 385
n_coincident 410
n_coincident 353
n_coincident 394
n_coincident 368
n_coincident 357
n_coincident 361
n_coincident 387
n_coincident 347
n_coincident 339
n_coincident 369
n_coincident 351
n_coincident 358
n_coincident 382
n_coincident 374
n_coincident 356
n_coincident 361
n_coincident 342
n_coincident 359
n_coincident 348
n_coincident 346
n_coincident 340
n_coincident 375
n_coincident 400
n_coincident 373
n_coincident 390
n_coincident 343
n_coincident 381
n_coincident 357
n_coincident 398
n_coincident 396
n_coincident 397
n_coincident 361
n_coincident 3

KeyboardInterrupt: 